In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
import re
from nltk import word_tokenize
from nltk.corpus import stopwords


In [ ]:
df = pd.read_csv("../data/jigsaw-toxic-comment-classification-challenge/train.csv")
df.head()

In [ ]:
len(df)

In [ ]:
print("report of the data composition")
print(f" Toxic data Composition \n{df['toxic'].value_counts()/len(df['toxic'])}\n\n")
print(f" severe_toxic data Composition \n{df['severe_toxic'].value_counts()/len(df['severe_toxic'])}\n\n")
print(f" obscene data Composition \n{df['obscene'].value_counts()/len(df['obscene'])}\n\n")
print(f" threat data Composition \n{df['threat'].value_counts()/len(df['threat'])}\n\n")
print(f" insult data Composition \n{df['insult'].value_counts()/len(df['insult'])}\n\n")
print(f" identity_hate data Composition \n{df['identity_hate'].value_counts()/len(df['identity_hate'])}\n\n")

In [ ]:
# df.drop(df[(df.toxic == 0 ) &	(df.severe_toxic == 0 ) & (df.obscene ==0)& (df.threat ==0)&(df.insult ==0)& (df.identity_hate == 0)].index, inplace=True)

In [ ]:
# print("report of the data composition")
# print(f" Toxic data Composition \n{df['toxic'].value_counts()/len(df['toxic'])}\n\n")
# print(f" severe_toxic data Composition \n{df['severe_toxic'].value_counts()/len(df['severe_toxic'])}\n\n")
# print(f" obscene data Composition \n{df['obscene'].value_counts()/len(df['obscene'])}\n\n")
# print(f" threat data Composition \n{df['threat'].value_counts()/len(df['threat'])}\n\n")
# print(f" insult data Composition \n{df['insult'].value_counts()/len(df['insult'])}\n\n")
# print(f" identity_hate data Composition \n{df['identity_hate'].value_counts()/len(df['identity_hate'])}\n\n")

In [ ]:
print("Toxic:",df[df['toxic']== 1 ]['comment_text'].values[random.randint(0,int(len(df)/10))])
print("Not Toxic :",df[df['toxic']== 0 ]['comment_text'].values[random.randint(0,int(len(df)/10))])

In [ ]:
stops=set(stopwords.words('english'))

In [ ]:
def textPreprocess(text):
    cleanText = text.strip()
    cleanText = text.lower()
    cleanText = word_tokenize(cleanText)
    cleanText = [word for word in cleanText if word not in stops]
    cleanText = " ".join(cleanText)
    cleanText = cleanText.replace('\n'," ")
    cleanText = re.sub(r"(?:^|\W)utc(?:$|\W)",'',cleanText)
    cleanText = re.sub(r'http[s]*://[A-Za-z0-9:./?=]*','url',cleanText)
    cleanText = re.sub(r"[^a-z\s]",'',cleanText)
    cleanText = re.sub(r'[\s]+'," ",cleanText)
    return cleanText

In [ ]:
textPreprocess("Cause i done CAre when i am with my baby yeah All the bad things disappear and you have been making me feel like i am loved by someday")

In [ ]:
textPreprocess(df.comment_text[1])


In [14]:
df['comment_text']=df['comment_text'].apply(lambda x: textPreprocess(x))

In [15]:
df=df[df['comment_text']!=""]

In [16]:
len(df)

159558

In [17]:
X=['comment_text']
y=["toxic",	"severe_toxic",	"obscene",	"threat",	"insult",	"identity_hate"]
X = pd.DataFrame(df['comment_text'])
Y = df.drop(columns=['comment_text','id'])

In [18]:
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
import re
import string

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [19]:
def getGloveCorpus(dims=300):
    # Set path and load corpus
    path = '../data/embedings/'
    filename = f'glove.6B.{dims}d.txt'
    f = open(path+filename, 'r', encoding='latin2')
    vec_txt = f.read()

    vec_data = {}
    words = vec_txt.split('\n')
    for word in words:
        vec = word.split()
        if len(vec) == dims+1:
            vec_data[vec[0]] = np.array([np.float16(x) for x in vec[1:]])
    vec = pd.DataFrame(vec_data, columns=None).transpose()
    return vec


def getGloveVec(word, vec, dims=300):
    vc = np.zeros(dims)
    try:
        vc = np.array(vec.loc[word])
    except:
        vc = np.zeros(dims)
    return vc


def getDocVec(sentence, vec, dims):
    tokens = word_tokenize(sentence)
    vecs = np.zeros(dims)
    for word in tokens:
        vecs += getGloveVec(word, vec, dims)
    vecs /= len(tokens)
    return vecs


def encodeClass(Y):
    label = []
    classes = Y.unique()
    for i in Y:
        label.append(np.where(classes == i)[0][0])
    return label


def getVecForm(X, Y, dims, vec, reshaping=None):
    '''
    X - Array of sentences.
    Y - Array of output class (numeric).
    dims - Vector dimensions (needed to pad correctly).
    '''
    vecList = []
    for i in X:
        vecList.append(getDocVec(i, vec, dims))
    X = np.asarray(vecList).astype(np.float16)
    Y = np.asarray(Y).astype(np.float16)
    if reshaping != None:
        X = np.reshape(X, reshaping)
    return X, Y


In [20]:
VEC = getGloveCorpus()

In [21]:

X_, Y_ = getVecForm(
    X = X.comment_text,
    Y = Y,
    vec = VEC,
    dims=300
)

/var/folders/0s/6jmf4pkj4yd7v5smf85g9mt00000gn/T/ipykernel_50094/2683019980.py:32: RuntimeWarning: invalid value encountered in true_divide
  vecs /= len(tokens)


In [22]:
X_ = np.reshape(X_, (X_.shape[0], 1, X_.shape[1]))
x_train, x_test, y_train, y_test = train_test_split(X_, Y_)

In [23]:
def simpleLSTM(X):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(8, input_shape=X.shape[1:]))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(6, activation='sigmoid'))

    opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt, metrics=['accuracy'])
    return model

In [24]:
X_.shape[1:]

(1, 300)

In [25]:
lstms = simpleLSTM(X_)
SPLIT = 0.25
EPOCH = 100
lstms.fit(x_train, y_train, validation_split=SPLIT, epochs=EPOCH)

2022-07-07 01:05:51.658445: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100
2805/2805 [==============================] - 6s 2ms/step - loss: nan - accuracy: 0.9850 - val_loss: nan - val_accuracy: 0.9943
Epoch 2/100
2805/2805 [==============================] - 5s 2ms/step - loss: nan - accuracy: 0.9943 - val_loss: nan - val_accuracy: 0.9943
Epoch 3/100
2805/2805 [==============================] - 3s 1ms/step - loss: nan - accuracy: 0.9943 - val_loss: nan - val_accuracy: 0.9943
Epoch 4/100
2805/2805 [==============================] - 3s 1ms/step - loss: nan - accuracy: 0.9943 - val_loss: nan - val_accuracy: 0.9943
Epoch 5/100
2805/2805 [==============================] - 3s 1ms/step - loss: nan - accuracy: 0.9943 - val_loss: nan - val_accuracy: 0.9943
Epoch 6/100
2805/2805 [==============================] - 4s 1ms/step - loss: nan - accuracy: 0.9943 - val_loss: nan - val_accuracy: 0.9943
Epoch 7/100
2805/2805 [==============================] - 3s 1ms/step - loss: nan - accuracy: 0.9943 - val_loss: nan - val_accuracy: 0.9943
Epoch 8/100
2805/2805 [====

In [26]:
lstms.evaluate(x_test, y_test)

1247/1247 [==============================] - 1s 485us/step - loss: nan - accuracy: 0.9939


[nan, 0.9938831925392151]

In [27]:
lstms.save("../model/lstmModel.h5")

In [28]:

lol = getDocVec(textPreprocess("This is a hate speech example "), VEC, 300)

lol = np.reshape([lol],(1,1,300))

lstms.predict(lol)


array([[nan, nan, nan, nan, nan, nan]], dtype=float32)